In [1]:
import os
import pyspark
from pyspark.sql import SparkSession

In [2]:
## URIs
NESSIE_URI = os.environ.get("NESSIE_URI")

## MINIO KEys
MINIO_REGION = os.environ.get("MINIO_REGION")
AWS_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY")
AWS_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY")
AWS_S3_ENDPOINT = "http://minio:9000"

## BUCKETS
BRONZE = os.environ.get("BRONZE")
SILVER = os.environ.get("SILVER")
GOLD = os.environ.get("GOLD")

In [3]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set(
            "spark.jars.packages",
            "software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178,org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.4.2,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.2_2.12:0.74.0",
        )
        .set("spark.sql.execution.pyarrow.enabled", "true")
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', AWS_S3_ENDPOINT)
        .set('spark.sql.catalog.nessie.warehouse', BRONZE)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_KEY)
        .set(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions",
        )
)

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-51454b7e-751a-48bb-b244-900e12407e42;1.0
	confs: [default]
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found software.amazon.awssdk#annotations;2.17.178 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found software.amazon.awssdk#http-client-spi;2.17.178 in central
	found sof

In [5]:
spark.sql("CREATE TABLE nessie.names (name STRING) USING iceberg;").show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


++
||
++
++



In [6]:
spark.sql("INSERT INTO nessie.names VALUES ('Guilherme Borba')")

DataFrame[]

In [7]:
spark.sql("SELECT * FROM nessie.names;").show()

+---------------+
|           name|
+---------------+
|Guilherme Borba|
+---------------+

